## Datasets utilisés
- "data_for_viz/consolidation-etalab-irve.csv": comporte les données des IRVE (https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/)
- "data_for_viz/voitures-rechargeables-par-commune-enrichies.csv": comporte le nombre de voitures immatriculées par commune et par type de charge (https://www.data.gouv.fr/fr/datasets/voitures-particulieres-immatriculees-par-commune-et-par-type-de-recharge/), données enrichies avec le département (https://geo.api.gouv.fr/decoupage-administratif/communes).
- "data_for_viz/communes-departement-region.csv": dataset comportant les communes de France, avec pour chacune leur département et région
- "data_for_viz/departements.geojson": comporte la géométrie des départements (https://france-geojson.gregoiredavid.fr/)

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

## Data prep IRVE

In [2]:
# Lectrure des données des irve
irve = pd.read_csv('data_for_viz/consolidation-etalab-irve.csv', sep=',', on_bad_lines='skip', dtype=str)
types_dict = {
    'nbre_pdc': int, 
    'puissance_nominale': float, 
    'consolidated_longitude': float,
    'consolidated_latitude': float,
    'consolidated_is_lon_lat_correct': bool,
    'consolidated_is_code_insee_verified': bool
              }
for col, col_type in types_dict.items():
    irve[col] = irve[col].astype(col_type)

In [3]:
# Lecture des données geojson des départements
geojson_dep = gpd.read_file(r'data_for_viz/departements.geojson')

In [4]:
# Lecture des données des communes
df_communes=pd.read_csv('data_for_viz/communes-departement-region.csv', dtype=str)

# Nettoyage des données des communes

# Modifier les codes communes INSEE, codes postaux, codes départements (ajout du 1er 0 manquant)
df_communes.loc[df_communes['code_commune_INSEE'].str.len() == 4, 'code_commune_INSEE'] = '0' + df_communes['code_commune_INSEE']
df_communes.loc[df_communes['code_postal'].str.len() == 4, 'code_postal'] = '0' + df_communes['code_postal']
df_communes.loc[df_communes['code_departement'].str.len() == 1, 'code_departement'] = '0' + df_communes['code_departement']
df_communes = df_communes[['code_commune_INSEE', 'code_departement', 'nom_departement']]

# Supprimer les codes INSEE en double
df_communes = df_communes.drop_duplicates(subset='code_commune_INSEE')

In [5]:
# Merge les irve avec les données des communes
irve = irve.merge(df_communes, left_on='code_insee_commune', right_on='code_commune_INSEE', how='left')

# regrouper les points de charge par localisation (couple lat/long)
pdc_par_loc = irve.groupby(['consolidated_longitude', 'consolidated_latitude']).agg({
    'nbre_pdc': 'count',
    'code_departement': lambda x: pd.Series.mode(x)
}).reset_index()
pdc_par_loc

# pour les bornes où il n'y a pas de code département, 
# identifier à partir de la latitude et longitude à quel département la borne appartient
def find_dep(dep):
    if isinstance(dep, str):
        return dep
    return np.nan

pdc_par_loc['code_departement'] = pdc_par_loc['code_departement'].apply(find_dep)

# pour les plus de 5000 points où le département n'a pas été identifié, 
# chercher le département correspondant à partir du polygone

# transformer le couple (lat, long) en point géométrique
pdc_par_loc['point'] = gpd.GeoSeries.from_xy(pdc_par_loc['consolidated_longitude'], 
                                             pdc_par_loc['consolidated_latitude'])
pdc_par_loc = gpd.GeoDataFrame(pdc_par_loc, geometry='point')

# mettre le code du département comme index pour le DataFrame geopandas des departements
geojson_dep = geojson_dep.set_index('code')

# sélectionner les points de charge où le département est manquant
missing_dep = pdc_par_loc[pdc_par_loc['code_departement'].isna()]
# Convertir la colonne 'point' en objet "Point" Shapely
missing_dep['geometry'] = missing_dep['point'].apply(Point)

# Pour chaque pdc où le département est manquant
for i, point in missing_dep.iterrows():
    # trouver l'index (code du département) des polygones du DataFrame geopandas des departments qui comportent le point
    polygons_containing_point = geojson_dep[geojson_dep['geometry'].contains(point['geometry'])].index.tolist()
    # Ajouter le code du département trouvé (si un département a été trouvé)
    for polygon_index in polygons_containing_point:
        pdc_par_loc.loc[i, 'code_departement'] = polygon_index
        break

# filter les départements: garder que la France métropolitaine pour la visu
pdc_par_loc = pdc_par_loc[pdc_par_loc['code_departement'].astype(str).apply(len)<3]

# Exporter le fichier pour la visualisation
pdc_par_loc.to_csv('data_for_viz/irve_par_loc.csv', index=False)

c:\Users\108739706\AppData\Local\pypoetry\Cache\virtualenvs\base-irve-project-8_pYZjG--py3.11\Lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [6]:
# Convertir les colonnes concernées en booléens
map_to_bool = {
    'false': False,
    '0': False,
    'FALSE': False,
    'False': False,
    'true': True,
    'TRUE': True,
    '1': True,
    'True': True
}

bool_columns = ['prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 'prise_type_autre',
               'gratuit', 'paiement_acte', 'paiement_cb', 'paiement_autre', 'reservation', 'station_deux_roues']

for col in bool_columns:
    irve[col] = irve[col].map(map_to_bool)

# exporter les données 'nettoyées'
irve.to_csv('data_for_viz/consolidation-etalab-irve-clean.csv')

In [7]:
# Créer un timeseries du nombre de stations de recharge au cours du temps

# remplacer les dates aberrantes par des valeurs nulles
irve.loc[(irve['date_mise_en_service']>'2024') | (irve['date_mise_en_service']<'2001'), 
         'date_mise_en_service'] = np.nan

# convertir la colonne en datetime
irve['date_mise_en_service'] = pd.to_datetime(irve['date_mise_en_service'], format="%Y-%m-%d")

# grouper par mois, appliquer une somme cumulative sur le nombre de stations et de points de charge
irve.set_index('date_mise_en_service', inplace=True)
irve_ts = (irve.groupby(pd.Grouper(freq="M")).nbre_pdc
               .agg(['count', 'sum'])
               .add_prefix('nbre_pdc_')
               .cumsum()
               .reset_index())

irve_ts = (irve.groupby(pd.Grouper(freq="M")).agg({
    'id_station_itinerance': 'nunique',
    'id_pdc_itinerance': 'nunique'
}).cumsum().reset_index()).rename(columns={'id_station_itinerance': 'nb_stations',
                                           'id_pdc_itinerance': 'nb_pdc'})

# exporter le fichier
irve_ts.to_csv('data_for_viz/irve_time_series.csv')

## Data prep immatriculations

In [8]:
# Lire le fichier des immatriculations
immatriculations = pd.read_csv('data_for_viz/voitures-rechargeables-par-commune-enrichies.csv',
                               dtype={'code_departement': 'str', 'code_region': 'str', 'siren': 'str'})

# Regrouper les données par date et département
immat_par_dep = (immatriculations.groupby(['date_arrete', 'code_departement', 'nom_departement'])
                                 .agg({'nb_vp': sum, 
                                       'nb_vp_rechargeables_el': sum,
                                       'nb_vp_rechargeables_gaz': sum,
                                       'population': sum}).reset_index())
immat_par_dep

# Ajouter la colonne nb_vp_non_rechargeables = nb_vp - nb_vp_rechargeables_el - nb_vp_rechargeables_gaz
immat_par_dep['nb_vp_non_rechargeables'] = (immat_par_dep['nb_vp'] -
                                            immat_par_dep['nb_vp_rechargeables_el'] - 
                                            immat_par_dep['nb_vp_rechargeables_gaz'])

# merger les données des coordonnées géographiques des départements avec les données des immatriculations
immat_par_dep = geojson_dep.reset_index()[['code', 'geometry']].merge(immat_par_dep, left_on="code", right_on="code_departement", how="inner")
immat_par_dep.head()

# exporter le fichier
immat_par_dep.to_csv('data_for_viz/immatriculations_par_dep.csv')

C:\Users\108739706\AppData\Local\Temp\ipykernel_18288\1713760436.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  immatriculations = pd.read_csv('data_for_viz/voitures-rechargeables-par-commune-enrichies.csv',


## Data prep graphiques points de charge

In [9]:
# Lire les données des irve
pdc_par_loc = pd.read_csv('data_for_viz/irve_par_loc.csv')
# regrouper les données des points de charge par département
pdc_par_dep = pdc_par_loc.groupby('code_departement').agg({'nbre_pdc': 'count'}).reset_index()

# Lire les données des immatriculations
immat_par_dep = pd.read_csv('data_for_viz/immatriculations_par_dep.csv', index_col=0)
# Conserver seulement les données du dernier trimestre 2022 (les plus récentes)
immat_par_dep_2022_12 = immat_par_dep[immat_par_dep['date_arrete']=='2022-12-31']

# Merge les deux jeux de données
pdc_immat_par_dep = pdc_par_dep.merge(immat_par_dep_2022_12, on='code_departement', how='inner')

# Calculer le nombre de points de charge par voiture électrique (pour chaque département) 
pdc_immat_par_dep['pdc_par_vp_el'] = pdc_immat_par_dep['nbre_pdc']/pdc_immat_par_dep['nb_vp_rechargeables_el']
# Calculer le nombre de points de charge par 1000 habitants (pour chaque département) 
pdc_immat_par_dep['pdc_par_1k_hab'] = 1000*pdc_immat_par_dep['nbre_pdc']/pdc_immat_par_dep['population']

# exporter le fichier
pdc_immat_par_dep.to_csv('data_for_viz/pdc_immat_par_dep.csv', index=False)